In [1]:
# Initial imports
import os
import pandas as pd
import nltk as nltk
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()

%matplotlib inline


[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\prabh\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [5]:
# Fetch the Oil news articles
#Crude_news_articles = news_api.get_everything(q = "Crude" and "Canada", language = "en")
from pathlib import Path
file_path = Path('headlines\combined_csv.csv')
Crude_news_articles = pd.read_csv(file_path)


In [6]:
Crude_news_article = Crude_news_articles.set_index('date')
Crude_news_article.head()

,headline,doc_type,material_type,section,keywords
date,,,,,
2021-10-01,When the Doctor’s Notes Hurt Your Feelings,article,News,NaN,"['Electronic Health Records', 'Doctors', 'inte..."
2021-10-01,House Delays Vote on Infrastructure Bill as De...,article,News,NaN,"['Infrastructure (Public Works)', 'American Jo..."
2021-10-01,Tirzah’s Genre-Less Pop Embraces the Beauty of...,article,Review,NaN,['Music']
2021-10-01,Senate Confirms Biden’s Pick to Lead the Burea...,article,News,NaN,"['Global Warming', 'Greenhouse Gas Emissions',..."
2021-10-01,UMass Amherst Hires Cybersecurity Firm to Inve...,article,News,NaN,"['Colleges and Universities', 'Discrimination'..."


In [7]:
crude_news_article = Crude_news_article[Crude_news_article["keywords"].str.contains("Petroleum")==True]
pd.set_option('display.max_colwidth', None)
crude_news_article.head()

,headline,doc_type,material_type,section,keywords
date,,,,,
2021-10-01,Senate Confirms Biden’s Pick to Lead the Bureau of Land Management,article,News,NaN,"['Global Warming', 'Greenhouse Gas Emissions', 'Federal Lands', 'Oil (Petroleum) and Gasoline', 'Drilling and Boring']"
2021-10-01,Britain Is Heading Into a Nightmarish Winter,article,Op-Ed,NaN,"['Great Britain Withdrawal from EU (Brexit)', 'Politics and Government', 'Shortages', 'Oil (Petroleum) and Gasoline', 'Trucks and Trucking', 'Labor and Jobs', 'Coronavirus (2019-nCoV)', 'Foreign Workers']"
2021-10-02,Harvard and Other Schools Make a Choice on Fossil Fuels,article,Op-Ed,NaN,"['Global Warming', 'Greenhouse Gas Emissions', 'Endowments', 'Colleges and Universities', 'Oil (Petroleum) and Gasoline']"
2021-10-03,‘Major’ Oil Spill Off California Coast Threatens Wetlands and Wildlife,article,News,NaN,"['Orange County, Calif, Oil Spill (2021)', 'Oil (Petroleum) and Gasoline', 'Oil Spills', 'Offshore Drilling and Exploration', 'Pipelines']"
2021-10-04,Oil prices hit a seven-year high as OPEC and its allies stick with a modest increase.,article,News,NaN,"['Oil (Petroleum) and Gasoline', 'Prices (Fares, Fees and Rates)']"


In [8]:
crude_news_article = crude_news_article.drop(['doc_type',"material_type",'section', 'keywords' ],axis=1)

In [9]:
# Sentiment score columns
crude_df = crude_news_article
crude_df["compound"] = 0.0000
crude_df["positive"] = 0.0000
crude_df["neutral"] = 0.0000
crude_df["negative"] = 0.0000
crude_df.head()


,headline,compound,positive,neutral,negative
date,,,,,
2021-10-01,Senate Confirms Biden’s Pick to Lead the Bureau of Land Management,0.0,0.0,0.0,0.0
2021-10-01,Britain Is Heading Into a Nightmarish Winter,0.0,0.0,0.0,0.0
2021-10-02,Harvard and Other Schools Make a Choice on Fossil Fuels,0.0,0.0,0.0,0.0
2021-10-03,‘Major’ Oil Spill Off California Coast Threatens Wetlands and Wildlife,0.0,0.0,0.0,0.0
2021-10-04,Oil prices hit a seven-year high as OPEC and its allies stick with a modest increase.,0.0,0.0,0.0,0.0


In [10]:
# Get sentiment for the text and the title
for index, row in crude_news_article.iterrows():
    try:
        # Sentiment scoring with VADER
        title_sentiment = analyzer.polarity_scores(row["headline"])
        crude_df["compound"][index] = title_sentiment["compound"]
        crude_df["positive"][index] = title_sentiment["pos"]
        crude_df["neutral"][index] = title_sentiment["neu"]
        crude_df["negative"][index] = title_sentiment["neg"]
        
    except AttributeError:
        pass
 
crude_df.head()



C:\Users\prabh\anaconda3\envs\alpacaenv\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\prabh\anaconda3\envs\alpacaenv\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
C:\Users\prabh\anaconda3\envs\alpacaenv\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\prabh\anaconda3\envs\alpacaenv

,headline,compound,positive,neutral,negative
date,,,,,
2021-10-01,Senate Confirms Biden’s Pick to Lead the Bureau of Land Management,0.0000,0.0,1.000,0.000
2021-10-01,Britain Is Heading Into a Nightmarish Winter,0.0000,0.0,1.000,0.000
2021-10-02,Harvard and Other Schools Make a Choice on Fossil Fuels,0.0000,0.0,1.000,0.000
2021-10-03,‘Major’ Oil Spill Off California Coast Threatens Wetlands and Wildlife,-0.3818,0.0,0.776,0.224
2021-10-04,Oil prices hit a seven-year high as OPEC and its allies stick with a modest increase.,-0.5574,0.0,0.735,0.265


In [11]:
mean_crude_df = crude_df.groupby(crude_df.index).mean()

In [12]:
mean_crude_df.to_csv( "oil_sentiments.csv")

In [13]:
mean_crude_df.describe()

,compound,positive,neutral,negative
count,179.000000,179.000000,179.000000,179.000000
mean,-0.101658,0.057844,0.817335,0.124832
std,0.340041,0.106161,0.169368,0.156445
min,-0.880700,0.000000,0.329000,0.000000
25%,-0.361200,0.000000,0.700000,0.000000
50%,0.000000,0.000000,0.813000,0.000000
75%,0.000000,0.037500,1.000000,0.236500
max,0.659700,0.425000,1.000000,0.671000
